In [1]:
"""Classes for the simulation of flat-fading channels"""

import tensorflow as tf
from sionna.channel import AWGN
from sionna.utils import complex_normal
class ApplyFlatFadingChannel(tf.keras.layers.Layer):
    # pylint: disable=line-too-long
    r"""ApplyFlatFadingChannel(add_awgn=True, dtype=tf.complex64, **kwargs)

    Applies given channel matrices to a vector input and adds AWGN.

    This class applies a given tensor of flat-fading channel matrices
    to an input tensor. AWGN noise can be optionally added.
    Mathematically, for channel matrices
    :math:`\mathbf{H}\in\mathbb{C}^{M\times K}`
    and input :math:`\mathbf{x}\in\mathbb{C}^{K}`, the output is

    .. math::

        \mathbf{y} = \mathbf{H}\mathbf{x} + \mathbf{n}

    where :math:`\mathbf{n}\in\mathbb{C}^{M}\sim\mathcal{CN}(0, N_o\mathbf{I})`
    is an AWGN vector that is optionally added.


    Parameters
    ----------
    add_awgn: bool
        Indicates if AWGN noise should be added to the output.
        Defaults to `True`.

    dtype : tf.complex64, tf.complex128
        The dtype of the output. Defaults to `tf.complex64`.

    Input
    -----
    (x, h, no) :
        Tuple:

    x : [batch_size, num_tx_ant], tf.complex
        Tensor of transmit vectors.

    h : [batch_size, num_rx_ant, num_tx_ant], tf.complex
        Tensor of channel realizations. Will be broadcast to the
        dimensions of ``x`` if needed.

    no : Scalar or Tensor, tf.float
        The noise power ``no`` is per complex dimension.
        Only required if ``add_awgn==True``.
        Will be broadcast to the shape of ``y``.
        For more details, see :class:`~sionna.channel.AWGN`.

    Output
    ------
    y : [batch_size, num_rx_ant, num_tx_ant], ``dtype``
        Channel output.
    """
    def __init__(self, add_awgn=True, dtype=tf.complex64, **kwargs):
        super().__init__(trainable=False, dtype=dtype, **kwargs)
        self._add_awgn = add_awgn

    def build(self, input_shape): #pylint: disable=unused-argument
        if self._add_awgn:
            self._awgn = AWGN(dtype=self.dtype)

    def call(self, inputs):
        if self._add_awgn:
            x, h, no = inputs
        else:
            x, h = inputs

        x = tf.expand_dims(x, axis=-1)
        y = tf.matmul(h, x)
        y = tf.squeeze(y, axis=-1)

        if self._add_awgn:
            y = self._awgn((y, no))

        return y

2024-07-12 21:14:52.642207: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 21:14:52.642243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 21:14:52.643906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 21:14:52.654129: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 21:14:53.820999: W tensorflow/compiler/tf2

: 